# Totale

##### setup

In [10]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf

username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
             complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
         complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
         complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
         complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
            pathPart1 = "/Users/"
            pathPart2 = "/documents/Github/SCGProject/Datasets/"
            complePath = pathPart1+username+pathPart2

            dfCambio = pd.read_csv(
                complePath+"/CorrectedDatasets/dfCambio.csv")

            dfClienti = pd.read_csv(
                complePath+"/CorrectedDatasets/dfClienti.csv")

            dfConsumi = pd.read_csv(
                complePath + "/CorrectedDatasets/dfConsumi.csv")

            dfCostoOrarioConsuntivo = pd.read_csv(
                complePath+"/CorrectedDatasets/dfConsuntivo.csv")

            dfCostoOrarioBudget = pd.read_csv(
                complePath+"CorrectedDatasets/dfCostoOrario.csv")

            dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

            dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

            dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

elif platform.system() == "Linux":
    complePath = "/home/alinux/unibg/git_repo/SCGProject"
    pathPart1 = "/Users/"
    pathPart2 = "/documents/Github/SCGProject/Datasets/"
    complePath = pathPart1+username+pathPart2

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")



dfCambio["TassoCambioMedio"] = dfCambio["TassoCambioMedio"].str.replace(',','.')


Username:  davidguzman


# Ricavi

In [11]:
dfVendite.rename(columns = {'budget/cons':'budget'}, inplace = True)
venditeConsuntivo = sqldf("SELECT DISTINCT * FROM dfVendite WHERE budget = 'CONSUNTIVO' or budget = 'Consuntivo'")
venditeBudget = sqldf("SELECT DISTINCT * FROM dfVendite WHERE budget = 'BUDGET' or budget = 'Budget'")

In [12]:
# join with dfCambio and dfClienti
venditeConsuntivo = sqldf('''SELECT DISTINCT v.NrMovimento, v.budget, v.NrArticolo, v.Quantity, v.ImportoVenditaValutaLocaleTOTALEVENDITA,k.TassoCambioMedio,v.ImportoVenditaValutaLocaleTOTALEVENDITA/k.TassoCambioMedio as TotaleEuro, v.NrOrigine, c.CodCondizioniPagam, c.FattCumulative, c.Valuta
FROM venditeConsuntivo as v join dfClienti as c on v.NrOrigine = c.Nr join dfCambio as k on c.Valuta = k.CodiceValuta
where k.Anno = 'Consuntivo' or k.Anno = 'CONSUNTIVO'
''')

venditeBudget = sqldf('''SELECT DISTINCT v.NrMovimento, v.budget, v.NrArticolo, v.Quantity, v.ImportoVenditaValutaLocaleTOTALEVENDITA,k.TassoCambioMedio,v.ImportoVenditaValutaLocaleTOTALEVENDITA/k.TassoCambioMedio as TotaleEuro, v.NrOrigine, c.CodCondizioniPagam, c.FattCumulative, c.Valuta
FROM venditeBudget as v join dfClienti as c on v.NrOrigine = c.Nr join dfCambio as k on c.Valuta = k.CodiceValuta
where k.Anno = 'Budget' or k.Anno = 'BUDGET'
''')

In [13]:
sumVenditeConsuntivo = sqldf('select sum(TotaleEuro) as sumVenditaTotale from venditeConsuntivo')
sumVenditeBudget = sqldf('select sum(TotaleEuro) as sumVenditaTotale from venditeBudget')
print(sumVenditeConsuntivo)
print(sumVenditeBudget)
print(sumVenditeConsuntivo-sumVenditeBudget) #questo è lo scostamento totale dei ricavi

   sumVenditaTotale
0     507842.851413
   sumVenditaTotale
0     395335.892443
   sumVenditaTotale
0      112506.95897


# Costi

Materie prime

In [14]:
dfConsumi.rename(columns = {'Budget/cons':'budget'}, inplace = True)
consumiConsuntivo = sqldf("SELECT DISTINCT * FROM dfConsumi WHERE budget = 'CONSUNTIVO'")
consumiBudget = sqldf("SELECT DISTINCT * FROM dfConsumi WHERE budget = 'BUDGET'")
sumConsumiConsuntivo = sqldf('select sum(ImportoCostoTOTALE) from consumiConsuntivo')
sumConsumiBudget = sqldf('select sum(ImportoCostoTOTALE) from consumiBudget')
print(sumConsumiConsuntivo)
print(sumConsumiBudget)
print(sumConsumiConsuntivo-sumConsumiBudget) #questo è lo scostamento totale della materia prima

   sum(ImportoCostoTOTALE)
0                 289639.5
   sum(ImportoCostoTOTALE)
0                192906.47
   sum(ImportoCostoTOTALE)
0                 96733.03


Lavorazioni interne

In [15]:
impiegoConsuntivo = sqldf("SELECT DISTINCT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'CONSUNTIVO' or budgetConsuntivo = 'Consuntivo'")
#join con i costi orari
impiegoConsuntivo  = sqldf('''SELECT DISTINCT i.NrArticolo,i.budgetConsuntivo,i.NrOrdineProduzione,i.Descrizione,i.NrAreaProduzione,i.Risorsa,i.QuantitydiOutput,i.TempoRisorsa,c.CostoOrario
FROM impiegoConsuntivo as i join dfCostoOrarioConsuntivo as c 
on c.AreaProduzione = i.NrAreaProduzione and c.Risorsa = i.Risorsa''')
# ci sono alcune quantità di output nulle, per gestire le sommiamo alle tuple con tutte le altre cose uguali
impiegoConsuntivo = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoConsuntivo
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')
impiegoConsuntivo

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario,QuantitydiOutput
0,ART0000018,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,15,2.0
1,ART0000018,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS5,1.0000,50,2.0
2,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0000,80,2.0
3,ART0000041,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,15,4.0
4,ART0000041,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS1,0.0001,80,4.0
...,...,...,...,...,...,...,...,...,...
3012,ART0005136,CONSUNTIVO,ODP19-06532,Fresatura,A30,RIS14,1.2500,50,3.0
3013,ART0005136,CONSUNTIVO,ODP19-06533,Controllo qualità,CQ,RIS16,0.0000,15,4.0
3014,ART0005136,CONSUNTIVO,ODP19-06533,Tornitura,A20,RIS13,0.5000,80,4.0
3015,ART0005136,CONSUNTIVO,ODP19-06688,Controllo qualità,CQ,RIS16,0.0000,15,9.0


In [16]:
impiegoBudget = sqldf("SELECT DISTINCT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'BUDGET' or budgetConsuntivo = 'Budget'")
#join con i costi orari
impiegoBudget  = sqldf('''SELECT DISTINCT i.NrArticolo,i.budgetConsuntivo,i.NrOrdineProduzione,i.Descrizione,i.NrAreaProduzione,i.Risorsa,i.QuantitydiOutput,i.TempoRisorsa,c.CostoOrario
FROM impiegoBudget as i join dfCostoOrarioConsuntivo as c 
on c.AreaProduzione = i.NrAreaProduzione and c.Risorsa = i.Risorsa''')
# righe a consuntivo sono di più che a budget e la somma dei 2 non corrisponde al numero di righe della tabella impiego risorse

# ci sono alcune quantità di output nulle, per gestire le sommiamo alle tuple con tutte le altre cose uguali
impiegoBudget = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoBudget
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')
impiegoBudget

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario,QuantitydiOutput
0,ART0000018,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,15,2.0
1,ART0000018,BUDGET,ODP17-05342,Fresatura,A30,RIS7,0.75,60,2.0
2,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.00,80,2.0
3,ART0000041,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,15,4.0
4,ART0000041,BUDGET,ODP17-05342,Fresatura,A30,RIS7,1.25,60,4.0
...,...,...,...,...,...,...,...,...,...
2006,ART0005136,BUDGET,ODP17-15077,Controllo qualità,CQ,RIS16,0.00,15,10.0
2007,ART0005136,BUDGET,ODP17-15077,Tornitura,A20,RIS14,1.25,60,10.0
2008,ART0005136,BUDGET,ODP17-16230,Brocciatura / stozzatrice,A33,RIS13,0.25,50,1.0
2009,ART0005136,BUDGET,ODP17-16230,Controllo qualità,CQ,RIS16,0.00,15,1.0


In [17]:
sumImpiegoConsuntivo = sqldf(
    'SELECT sum(TempoRisorsa*CostoOrario) FROM impiegoConsuntivo WHERE TempoRisorsa>=0')
sumImpiegoBudget = sqldf(
    'SELECT sum(TempoRisorsa*CostoOrario) FROM impiegoBudget WHERE TempoRisorsa>=0')
print(sumImpiegoConsuntivo)
print(sumImpiegoBudget)
print(sumImpiegoConsuntivo-sumImpiegoBudget)

   sum(TempoRisorsa*CostoOrario)
0                    359238.2962
   sum(TempoRisorsa*CostoOrario)
0                    228848.8276
   sum(TempoRisorsa*CostoOrario)
0                    130389.4686


In [30]:
print(sumVenditeConsuntivo.values[0][0])
print(sumImpiegoConsuntivo.values[0][0]+sumConsumiConsuntivo.values[0][0])
print(sumVenditeConsuntivo.values[0][0]-sumImpiegoConsuntivo.values[0][0]-sumConsumiConsuntivo.values[0][0])
print(sumVenditeBudget.values[0][0])
print(sumImpiegoBudget.values[0][0]+sumConsumiBudget.values[0][0])
print(sumVenditeBudget.values[0][0]-sumImpiegoBudget.values[0][0]-sumConsumiBudget.values[0][0])

507842.8514127266
648877.7962
-141034.94478727342
395335.8924431994
421755.29760000017
-26419.405156800756
